# SpikeInterface Tutorial -  GDR School Lyon - October 2021


In this practice, you will use SpikeInterface to analyze a 64-channel dataset from am "ASSY-156-P1" probe from Cambridge Neurotech. 
The dataset is kindly provided by [Samuel McKenzie's Lab](https://mckenzieneurolab.com/). 

The objective of this practice is for you to familiarize with SpikeInterface on a real-world example and to explore its capabilities and functionalities.

# Table of contents

* [0. Preparation](#preparation)
* [1. Loading the data and probe information](#loading)
* [2. Preprocessing](#preprocessing)
* [3. Saving and loading SpikeInterface objects](#save-load)
* [4. Spike sorting](#spike-sorting)
* [5. Extracting waveforms](#waveforms)
* [6. Postprocessing](#postprocessing)
* [7. Viewers](#viewers)
* [8. Validation and curation](#curation)
* [9. Spike sorting comparison](#comparison)


# 0. Preparation <a class="anchor" id="preparation"></a>

### Download the ephys data

First, we need to download the recording (you should have this already). Feel free to use your own recordings as well later on. 

The dataset is called `cambridge_data.dat` and can be found on this [drive link](https://drive.google.com/drive/folders/1eWPuOd8q4MjpVpwazkWygQJDzJnToN3i) (`practice_2_real_dataset` folder). Move the dataset in the current folder.
The recording was performed with the "ASSY-156-P1" probe with 4 shanks of 16 channels (in total 64 channels).


### Import the modules

Let's now import the `spikeinterface` modules that we need:

In [ ]:
import spikeinterface as si
import spikeinterface.extractors as se 
import spikeinterface.toolkit as st
import spikeinterface.sorters as ss
import spikeinterface.comparison as sc
import spikeinterface.widgets as sw

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

%matplotlib notebook

# 1. Loading recording and probe information <a class="anchor" id="loading"></a>

In [ ]:
# file path
recording_file = 'cambridge_data.dat'

# parameters to load the bin/dat format
num_channels = 64
sampling_frequency = 20000
gain_to_uV = 0.195
offset_to_uV = 0
dtype = "int16"
time_axis = 0

In [ ]:
si.read_binary?

In [ ]:
recording = si.read_binary(recording_file, num_chan=num_channels, sampling_frequency=sampling_frequency,
                           dtype=dtype, gain_to_uV=gain_to_uV, offset_to_uV=offset_to_uV, 
                           time_axis=time_axis)

The `read_binary()` function returns a `RecordingExtractor` object. We can print it to visualize some of its properties:

In [ ]:
print(recording)

We can further `annotate` the recording to tell SI that it is not filtered yet. This will prevent further mistakes in the pipieline, such as attempting to extract waveforms from unfiltered data.

In [ ]:
recording.annotate(is_filtered=False)

While the `read_binary()` function is part of the `core` module (as it's used internally by SI to store data in a convenient format), the `extractor` module allows you to load many file formats used in electrophysiology. 

The extractors available in SI are all loaded using the [NEO](https://neo.readthedocs.io/en/stable/) python package.

A `RecordingExtractor` object extracts information about channel ids, channel locations (if present), the sampling frequency of the recording, and the extracellular traces (when prompted). The `BinaryRecordingExtractor` is designed specifically for raw binary files datasets (.bin, .dat, .raw).

Here we load information from the recording using the built-in functions from the RecordingExtractor

In [ ]:
channel_ids = recording.get_channel_ids()
fs = recording.get_sampling_frequency()
num_chan = recording.get_num_channels()
num_segments = recording.get_num_segments()

print(f'Channel ids: {channel_ids}')
print(f'Sampling frequency: {fs}')
print(f'Number of channels: {num_chan}')
print(f"Number of segments: {num_segments}")

### Exercise 1a) Explore traces

Explore the `recording.get_traces()` function. What are its arguments? What is the shape of the returned signals? What is the unit? 

Try to plot the traces of some channels using `matplotlib`.

In [ ]:
# explore the traces here

**IMPORTANT**: The `widgets` module includes several convenient plotting functions that can be used to explore the data:

In [ ]:
w_ts = sw.plot_timeseries(recording, channel_ids=[0, 1, 2, 3])

Before moving on with the analysis, we have to load the probe information. For this we will use the [ProbeInterface](https://probeinterface.readthedocs.io/en/main/index.html) package. 

ProbeInterface allows to easily create, manipulate, and visualize neural probes. Moreover, it comes with a wide range of IO functions to import and export existing formats. Finally, we have created a public library of commercial probes (https://gin.g-node.org/spikeinterface/probeinterface_library/) that can be retrieved with a single line of code.

Let's import `probeinterface`, download the probe and plot it!

In [ ]:
import probeinterface as pi
from probeinterface import plotting

In [ ]:
manufacturer = 'cambridgeneurotech'
probe_name = 'ASSY-156-P-1'

probe = pi.get_probe(manufacturer, probe_name)
print(probe)

In most experiments, the neural probe has a connector, that is interfaced to an headstage, which in turn connects to the acquisition system. This *pathway* usually results in a channel remapping, which means that the order of the contacts on the probe is different than the order of the recorded traces.

`probeinterface` provides a growing collection of common pathways that can be loaded directly to wire a device and apply the correct channel mapping:

In [ ]:
pi.get_available_pathways()

In [ ]:
probe.wiring_to_device('ASSY-156>RHD2164')

In [ ]:
plotting.plot_probe(probe, with_contact_id=True, with_device_index=True)

The probe now has contact ids `id#` and device ids `dev#`! We can also visualize the probe information as a `pandas` dataframe:

In [ ]:
probe.to_dataframe(complete=True).loc[:, ["contact_ids", "shank_ids", "device_channel_indices"]]

Note that also the `shank_id` is loaded with the probe.

In [ ]:
recording_prb = recording.set_probe(probe, group_mode="by_shank")

When loading the probe, the device indices (and all the other contact properties) are automatically sorted:

In [ ]:
probe_rec = recording_prb.get_probe()
probe_rec.to_dataframe(complete=True).loc[:, ["contact_ids", "shank_ids", "device_channel_indices"]]

In [ ]:
print(f'Channels after loading the probe file: {recording_prb.get_channel_ids()}')
print(f'Channel groups after loading the probe file: {recording_prb.get_channel_groups()}')

### Properties 

`RecordingExtractor` object can have *properties*. A property is a piece of information attached to a channel, e.g. group or location.

Similarly, for `SortingExtractor` objects (that we'll cover later), anything related to a unit can be stored as a property. 

We can check which properties are in the extractor as follows:

In [ ]:
print("Properties before loading the probe:", list(recording.get_property_keys()))

In [ ]:
print("Properties after loading the probe:", list(recording_prb.get_property_keys()))

After loading the probe we now have some new properties: `contact_vector`, `location`, and `group`.

Let's add some new properties! 
The first 32 channels are in the CA1 area, the second 32 are in the CA3 area:

### Exercise 1b) Add a custom property

We can use the `recording.set_property()` function to add custom properties to a SpikeInterface object. Properties are useful to store pieces of information across the analysis. 

Try to add a custom property to your `recording_prb` object (e.g. brain area, impedance, etc.). Check that the property was correctly added once you're done.

In [ ]:
recording_prb.set_property?

In [ ]:
# Add your property(ies) here

### (Optional) 1c) Can you add a property to a subset of channels? 

In [ ]:
print("Properties after adding custom properties:", list(recording_prb.get_property_keys()))

# 2. Preprocessing <a class="anchor" id="preprocessing"></a>


Now that the probe information is loaded we can do some preprocessing using `toolkit` module.

We can filter the recordings, rereference the signals to remove noise, discard noisy channels, whiten the data, remove stimulation artifacts, etc. (more info [here](https://spiketoolkit.readthedocs.io/en/latest/preprocessing_example.html)).

For this notebook, let's filter the recordings and apply common median reference (CMR). All preprocessing modules return new `RecordingExtractor` objects that apply the underlying preprocessing function. This allows users to access the preprocessed data in the same way as the raw data.

We will focus only on the first shank (grouo `0`) for the following analysis:

In [ ]:
recordings_by_group = recording_prb.split_by("group")
print(recordings_by_group)

In [ ]:
group_to_sort = 2

In [ ]:
recording_to_process = recordings_by_group[2]

Below, we bandpass filter the recording:

In [ ]:
recording_f = st.bandpass_filter(recording_to_process, freq_min=300, freq_max=6000)

w = sw.plot_timeseries(recording_f)

We can see that the after filtering we can observe spiking activity on many channels! We can also apply other preprocessing steps to further increase the quality of the recording. 

### Exercise 2a) Explore noise reduction methods

We can use the `toolkit` module to further increase the quality of the traces before spike sorting.
Several available preprocessors are available for this purpose.

- Explore the `toolkit` module (`st.`) to find functions that could remove noise (e.g. by re-referencing the signal).

- Choose a preprocessing step and apply it to the filtered recording (`recording_f`)

- Compare the traces to the filtered traces

In [ ]:
# explore the toolkit module here

In [ ]:
# Use the right object to continue with the analysis
recording_processed = recording_f 

### Exercise 2b) Visual exploration preprocessing methods with `ephyviewer`

In [ ]:
import ephyviewer

In [ ]:
app = ephyviewer.mkQApp()
win = ephyviewer.MainViewer(debug=True, show_auto_scale=True)

In [ ]:
sig_source = ephyviewer.SpikeInterfaceRecordingSource(recording=recording_to_process)
sig_filtered_source = ephyviewer.SpikeInterfaceRecordingSource(recording=recording_f)

In [ ]:
view1 = ephyviewer.TraceViewer(source=sig_source, name='signals')
view1.params['scale_mode'] = 'same_for_all'
win.add_view(view1)

view2 = ephyviewer.TraceViewer(source=sig_filtered_source, name='signals filtered')
view2.params['scale_mode'] = 'same_for_all'
win.add_view(view2)

In [ ]:
# you can add more views here!

In [ ]:
win.show()
app.exec_()

Since we are going to spike sort the data, let's first cut out a 5-minute recording, to speed up computations.

We can easily do so with the `frame_slice()` function:

In [ ]:
fs = recording_processed.get_sampling_frequency()
recording_sub = recording_processed.frame_slice(start_frame=0*fs, end_frame=300*fs)

# 3. Saving and loading SpikeInterface objects <a class="anchor" id="save-load"></a>

All operations in SpikeInterface are *lazy*, meaning that they are not performed if not needed. This is why the creation of our filter recording was almost instantaneous. However, to speed up further processing, we might want to **save** it to a file and perform those operations (eg. filters, CMR, etc.) at once. 

In [ ]:
recording_saved = recording_sub.save(folder="preprocessed", progress_bar=True, 
                                     n_jobs=4, total_memory="100M")

The `save` returns a new *cached* recording that has all the previously loaded information:

In [ ]:
print(f'Cached channels ids: {recording_saved.get_channel_ids()}')
print(f'Channel groups after caching: {recording_saved.get_channel_groups()}')

After saving the SI object, we can easily load it back in a new session:

In [ ]:
recording_loaded = si.load_extractor("preprocessed/")

In [ ]:
print(f'Loaded channels ids: {recording_loaded.get_channel_ids()}')
print(f'Channel groups after loading: {recording_loaded.get_channel_groups()}')

We can double check that the traces are exactly the same as the `recording_saved` that we saved:

In [ ]:
fig, axs = plt.subplots(ncols=2)
w_saved = sw.plot_timeseries(recording_saved, ax=axs[0])
w_loaded = sw.plot_timeseries(recording_loaded, ax=axs[1])
axs[0].set_title("Saved")
axs[1].set_title("Loaded")

**IMPORTANT**: the same saving mechanisms are available also for all SortingExtractor

# 4. Spike sorting <a class="anchor" id="spike-sorting"></a>

We can now run spike sorting on the above recording.

Let's first check the installed sorters in SpikeInterface.
We will sort the bandpass cached filtered recording the `recording_saved` object.

In [ ]:
ss.installed_sorters()

We can retrieve the parameters associated to any sorter with the `get_default_params()` function from the `sorters` module:

In [ ]:
sorter_name = 'sorter-name-here'

In [ ]:
ss.get_default_params(sorter_name)

In [ ]:
ss.get_params_description(sorter_name)

### Exercise 4a) Spike sort your data

The `ss.run_sorter()` function from the `sorters` module is where the spike sorting happens! The function returns a `SortingExtractor` object, which allows you to retrieve spiking data (e.g. unit ids, spike trains, etc.)

Run a spike sorter installed on your system and explore the returned `SortingExtractor` object.

In [ ]:
ss.run_sorter?

To modify a parameter, we can easily pass it to the `run` function as an extra argument!
For example, let's set the `filter` parameter to False as the recording is already preprocessed:

In [ ]:
# run spike sorting on entire recording
sorting = ss.run_sorter(...)

In [ ]:
print(sorting)

sorting.?

### Exercise 4b) Change a spike sorter parameter

Parameters (listed above) can be directly passed as extra arguments to the `ss.run_sorter()` function.

Try to choose a parameter, change its default value, and run spike sorting again


In [ ]:
# run spike sorting on entire recording
sorting1 = ss.run_sorter(..., my_param1="")

# Other spike sorting options 

You don't need to run these, but they might be helpful for reference in the future :)

## 4.1 Install and run a MATLAB-based sorter (requires MATLAB)

For MATLAB-based sorters, all you need to do is cloning the sorter repo and point it to SpikeInterface:

Let's clone [`ironclust`](https://github.com/flatironinstitute/ironclust) in the current directory:

In [ ]:
!git clone https://github.com/flatironinstitute/ironclust

Now all we have to tell the IronClustSorter class where is the ironclust repo:

In [ ]:
ss.IronClustSorter.set_ironclust_path('./ironclust')

Note that we can also set a global environment variable called `IRONCLUST_PATH`. In that case we don't need to set the path in each session because the sorter class looks for this environment variable.

Now ironclust should be installed and we can run it:

In [ ]:
ss.IronClustSorter.ironclust_path

In [ ]:
ss.installed_sorters()

In [ ]:
# run spike sorting by group
sorting_IC = ss.run_sorter("ironclust", recording_saved, 
                           output_folder='results_IC',
                           verbose=True)
print(f'IronClust found {len(sorting_IC.get_unit_ids())} units')

The spike sorting returns a `SortingExtractor` object. Let's see some of its functions:

In [ ]:
print(f'Ironclust unit ids: {sorting_IC.get_unit_ids()}')

In [ ]:
print(f'Spike train of a unit: {sorting_IC.get_unit_spike_train(13)}')

We can use `spikewidgets` functions for some quick visualizations:

In [ ]:
w_rs = sw.plot_rasters(sorting_IC)

## 4.2 Running multiple sorter jobs in parallel

So far we have seen how to run one sorter at a time. SI provides a convenient launcher in order to run multiple sorters on multiple recordings with one line of code!

The `run_sorters()` function of the `sorter` module allows you to specify a list of sorters to use on a list (or dictionary) of parameters. The jobs are by default ran in a loop, but the `engine` argument enables to specify a parallel backend (`joblib` or `dask`) and relative parameters.

In the following example, we run the 2 jobs to run `herdingspikes` and `ironclust` in parallel:

In [ ]:
sorting_outputs = ss.run_sorters(sorter_list=["herdingspikes", "ironclust"],
                                 recording_dict_or_list={"group0": recording_saved},
                                 working_folder="all_sorters",
                                 verbose=False,
                                 engine="joblib",
                                 engine_kwargs={'n_jobs': 2})

The returned `sorting_outputs` variable is a dictionary that has (rec_name, sorter_name) as keys, and the `SortingExtractor` objects as valus:

In [ ]:
print(sorting_outputs.keys())

For the rest of the tutorial, let's pick the `ironclust` output:

In [ ]:
sorting_IC = sorting_outputs[('group0', 'ironclust')]

## 4.3 Spike sort in Docker containers
###  (Linux and MacOS only)

Some sorters are hard to install! To alleviate this headache, SI provides a built-in mechanism to run a spike sorting job in a docker container.

We are maintaining a set of sorter-specific docker files in the [spikeinterface-dockerfiles repo](<https://github.com/SpikeInterface/spikeinterface-dockerfiles>)
and most of the docker images are available on Docker Hub from the [SpikeInterface organization](<https://hub.docker.com/orgs/spikeinterface/repositories>).

Running spike sorting in a docker container just requires to:

1. have docker installed
2. have docker python SDK installed (`pip install docker`)

When docker is installed, you can simply run the sorter in a specified docker image:

In [ ]:
print(ss.installed_sorters())

In [ ]:
sorting_SC = ss.run_spykingcircus(recording_saved, output_folder="results_SC",
                                  docker_image="spikeinterface/spyking-circus-base:1.0.7", 
                                  verbose=True)

In [ ]:
sw.plot_rasters(sorting_SC)

# 5. Extracting waveforms <a class="anchor" id="waveforms"></a>

The core of postprocessing spike sorting results revolves around extracting waveforms from paired recording-sorting objects.

The `WaveformExtractor` object has convenient functions to retrieve waveforms and templates.
Let's see how it works.

To extract the waveforms, we can run:

In [ ]:
si.extract_waveforms?

In [ ]:
we = si.extract_waveforms(recording_saved, sorting, folder="waveforms_si", progress_bar=True,
                          n_jobs=1, total_memory="500M", overwrite=True)
print(we)

Now all waveforms are computed and stored in the provided `waveforms_si` folder. We can now retrieve waveforms and templates easily.

### Exercise 5a) Explore waveforms and templates

The `WaveformExtractor` object (`we`) has convenient functions to retrieve waveforms and templates (average waveforms).

- Retrieve some waveforms and templates and inspect them
- Are waveform extracted from all spikes? If not, can you explore the function docs and find a way to compute waveforms for all spikes?


In [ ]:
# explore waveforms and templates


### Exercise 5b) Find `widgets` for plotting waveforms

Use the `widgets` module to visualize waveform extractor objects

In [ ]:
# explore widgets here

# 6. Postprocessing <a class="anchor" id="postprocessing"></a>

Postprocessing spike sorting results ranges from computing additional information, such as spike amplitudes and Principal Component Analisys (PCA) scores, to computing features of the extracellular waveforms, similarity between templates and crosscorrelograms. All of this is possible with the `toolkit` module.

### PCA scores

PCA scores can be easily computed with the `compute_principal_components()` function. Similarly to the `extract_waveforms`, the function returns an object of type `WaveformPrincipalComponent` that allows to retrieve all pc scores on demand.

In [ ]:
pc = st.compute_principal_components(we, n_components=3)

In [ ]:
pc0 = pc.get_components(unit_id=0)
all_labels, all_pcs = pc.get_all_components()

For pc scores of a single unit, the dimension is (num_spikes, num_components, num_channels). 
The `get_all_components()` function returns an array with the label/unit id for each component (`all_labels`) and an array of dimension (num_all_samples, num_components, num_channels). 

**IMPORTANT**: The `get_components()` and `get_all_components()` will be renamed `get_projections()` and `get_all_projections()` in the next release.

### Spike amplitudes

Spike amplitudes can be computed with the `get_spike_amplitudes` function.

In [ ]:
amplitudes = st.get_spike_amplitudes(we, outputs="concatenated", progress_bar=True)

By default, all amplitudes are concatenated in one array.

The correspinding spike times and labels can be easily retrieved as:

In [ ]:
all_spike_times, all_spike_labels = sorting.get_all_spike_trains()[0]

The [0] index is to select the first segment. In case of multiple segments each element will correspond to a different segment and will contain spike times and labels for that segment.


### Exercise 6a) Plot pca projections and amplitudes over time

Explore the principal component space and amplitudes using `matplotlib`

- Plot projections of two units on multiple channels and pc components
- Plot the spike amplitudes of a unit over time

In [ ]:
# Explore pc and amplitudes

### Compute template metrics

Template metrics, or extracellular features, such as peak to valley duration or full-width half maximum, are important to classify neurons into putative classes (excitatory - inhibitory). The `toolkit` allows one to compute several of these metrics:

In [ ]:
print(st.get_template_metric_names())

In [ ]:
template_metrics = st.calculate_template_metrics(we)
display(template_metrics)

For more information about these template metrics, we refer to this [documentation](https://github.com/AllenInstitute/ecephys_spike_sorting/tree/master/ecephys_spike_sorting/modules/mean_waveforms) from the Allen Institute.

# 7. Viewers <a class="anchor" id="viewers"></a>

The `spikeinterface-gui` allows you to interactively explore a spike sorting output once waveforms are extracted.


### Exercise 7a) Use the `spikeinterface-gui` to epxlore the sorting output

Launch the `sigui` on the waveforms folder and explore all different views.

- Can you find some bad units?
- Are there missed spikes or spike sorting errors?

In [ ]:
!sigui waveforms_si

# 8. Validation and curation <a class="anchor" id="curation"></a>

The `toolkit` module also provides several functions to compute qualitity metrics to validate the spike sorting results.

Let's see what metrics are available:

In [ ]:
print(st.get_quality_metric_list())

In [ ]:
qc = st.compute_quality_metrics(we_all, waveform_principal_component=pc)

In [ ]:
display(qc)

For more information about these waveform features, we refer to this [documentation](https://allensdk.readthedocs.io/en/latest/_static/examples/nb/ecephys_quality_metrics.html) from the Allen Institute.

## Automatic curation based on quality metrics

A viable option to curate (or at least pre-curate) a spike sorting output is to filter units based on quality metrics. As we have already computed quality metrics a few lines above, we can simply filter the `qc` dataframe based on some thresholds.

Here, we'll only keep units with an SNR > 5 and an ISI violation threshold < 0.2:

In [ ]:
snr_thresh = 5
isi_viol_thresh = 0.2

A straightforward way to filter a pandas dataframe is via the `query`.
We first define our query (make sure the names match the column names of the dataframe):

In [ ]:
our_query = f"snr > {snr_thresh} & isi_violations_rate < {isi_viol_thresh}"
print(our_query)

and then we can use the query to select units:

In [ ]:
keep_units = qc.query(our_query)
keep_unit_ids = keep_units.index.values

In [ ]:
sorting_auto = sorting.select_units(keep_unit_ids)
print(f"Number of units before curation: {len(sorting.get_unit_ids())}")
print(f"Number of units after curation: {len(sorting_auto.get_unit_ids())}")

### 8a) Extract waveforms on the curated sorting and explore the output in the spikeinterface-gui

Re-extract waveforms using the curated output and explore the results.

Is the spike sorting quality improved?

In [ ]:
# choose a different folder!
we_curated = ...

In [ ]:
!sigui new_folder

### 8b) Explore different curation strategies in the spikeinterface-gui

- Try to use different thresholds and/or quality metrics to automatically curate the spike sorting output.
- View the result in the `spikeinterface-gui`

In [ ]:
# try different metrics/thresholds for curation

# 9. Exporters <a class="anchor" id="exporters"></a>

The `exporters` module provides functions to export the spike sorting results to different frameworks.

## Export to Phy for manual curation

To perform manual curation we can export the data to [Phy](https://github.com/cortex-lab/phy). 

In [ ]:
from spikeinterface.exporters import export_to_phy

In [ ]:
export_to_phy(we_curated, output_folder='phy',
              progress_bar=True, total_memory='100M')

In [ ]:
%%capture --no-display
!phy template-gui phy/params.py

After curating the results we can reload it using the `PhySortingExtractor` and exclude the units that we labeled as `noise`:

In [ ]:
sorting_phy_curated = se.PhySortingExtractor('phy/', exclude_cluster_groups=['noise'])

In [ ]:
print(f"Number of units before curation: {len(sorting.get_unit_ids())}")
print(f"Number of units after curation: {len(sorting_phy_curated.get_unit_ids())}")

## Export spike sorting report

The SpikeInterface report automatically saves summary and unit-specific views to save and inspect your spike sorting output.

In [ ]:
from spikeinterface.exporters import export_report

In [ ]:
export_report(waveform_extractor=we_all, output_folder="SI_report", format="pdf", 
              metrics=qc)